In [ ]:
import ipyparallel as ipp
cluster = ipp.Client()

In [ ]:
v = cluster[:]
lview = cluster.load_balanced_view()
len(v)

In [ ]:
%%px --local

import os
try:
    import mkl
    mkl.set_num_threads(1)
except:
    pass

os.environ['OPENBLAS_NUM_THREADS'] = "1"
os.environ['OMP_NUM_THREADS'] = "1"

 # Import necessary liblaries

In [ ]:
%%px --local

import scipy.linalg as la
import kwant
import tinyarray
import numpy as np
import matplotlib.pyplot as plt
import types
from scipy.constants import physical_constants
from scipy.signal import argrelextrema
import tinyarray
from ipywidgets import interact
plt.rcParams.update({'font.size': 25})

In [ ]:
import adaptive
adaptive.notebook_extension()
import pandas as pd

In [ ]:
%%px --local

val_hJ= 6.62607004e-34 #value in J
val_e = physical_constants['elementary charge'][0]
val_hbar = physical_constants['Planck constant over 2 pi in eV s'][0]
val_m0 = physical_constants['electron mass energy equivalent in MeV'][0]
Phi_0=2.067*1e-15 
c = physical_constants['speed of light in vacuum'][0]
val_m0 = val_m0 / (c*10**9)**2 * 10**6 # [eV * s**2 / nm**2]
m=0.014
val_hbarJ = physical_constants['Planck constant over 2 pi'][0]*1e18 # in nm
mi_b= physical_constants['Bohr magneton in eV/T'][0]

sigma_0 = tinyarray.array([[1,0],[0,1]])   
sigma_x = tinyarray.array([[0,1],[1,0]])
sigma_y = tinyarray.array([[0,-1.j],[1.j,0]])
sigma_z = tinyarray.array([[1,0],[0,-1]])

tau_0 = tinyarray.array([[1,0],[0,1]]) 
tau_x = tinyarray.array([[0, 1], [1, 0]])
tau_y = tinyarray.array([[0, -1j], [1j, 0]])
tau_z = tinyarray.array([[1, 0], [0, -1]])

trs_m = -1j*sigma_y 

In [ ]:
%%px --local

J_max=1
dx=10
par = types.SimpleNamespace(dx=dx,
                            L=200, #lenght
                            W=1000, #width
                            t=val_hbar**2/val_m0/m/dx**2/2, 
                            V_value=  1, #potential energy of a tip
                            tip=50, # effective size of tip potential
                            mi=40e-3, # chemical potential
                            B=0, #magnetic field in Z direction
                            phi=0, #phase
                            xt=0) # xtip position

In [ ]:
%%px --local

sc_flux_quantum = 2.067833848*1e-15
L = par.L*1e-9
W = par.W*1e-9
B_period = sc_flux_quantum/L/W*1e3

# Base supercurrent distribution with tip

In [ ]:
%%px --local 

def J_S0(x,par): # J_S is supercurrent distribution 
    if -par.W/2<=x<par.W/2 :
        return J_max- par.V_value/(1+( (x-par.xt)**2 )/par.tip**2)
    else:
        return 0

In [ ]:
x_tab = np.arange(-par.W/2-20,par.W/2+20,1)
I=[]
par.V_value=1
for x in x_tab:
    I.append( J_S0(x,par) )
    
fig, ax1 = plt.subplots(figsize=(10,5))
plt.plot(x_tab,I,'k',linewidth=4)
plt.fill(x_tab,I,'r')
plt.xlim(-520,520)
plt.grid()
plt.xlabel("$ x $ [nm]")
plt.ylabel(" $J$ $[J_0]$")
#plt.savefig("J0.pdf")

# gamma-- effective phase diffrence

In [ ]:
%%px --local

def gamma(x,par): # overall phase diffrence
    integral=0
    dy=par.dx
    for y in np.arange(0,par.L,dy) :
        integral+=  x*dy 
    return par.phi+par.B*integral *  2*np.pi/Phi_0 *1e-18  

In [ ]:
%%px --local

def curr_dist(x,par):
    return J_S0(x,par) * np.sin(gamma(x,par))

In [ ]:
%%px --local

def B_I_c(par,B): #
    # Given parameters par and magnetic field B, returns value of critical current 
    par.B=B
    I=[]
    for phi in np.linspace(0,2*np.pi,200):
        par.phi=phi
        value=0
        d=10
        for x in np.arange(-par.W/2,par.W/2,d):
            value+= curr_dist(x,par) *d
            
        I.append( value )
        Ic = max(I)
    return Ic

In [ ]:
par.V_value=0
out=[]
N =5 # N=100
phi_tab = np.linspace(0,2*np.pi,N)
B_tab=np.linspace(-50e-3,50e-3,N)
x_tab = np.arange(-par.W/2-1,par.W/2+1,1)

# For diffrent magnetic field, we find that phi value which gives critical current and then
# I save that curent distribution
for B in B_tab:
    par.B=B
    I=[]
    for phi in phi_tab:
        par.phi=phi
        values=[]

        for x in x_tab:
            values.append(curr_dist(x,par) )
        I.append(sum(values))


    par.phi=phi_tab[ np.argmax(I)]
    values=[]
    x_tab = np.arange(-par.W/2-1,par.W/2+1,1)
    for x in x_tab:
        values.append(curr_dist(x,par) )
    out.append(values)


 # overwrites data
pd.DataFrame(out ).to_pickle('analytics_supercurrent_distribution_map.pkl')    #to save the dataframe, df to *.pkl



In [ ]:
out  = pd.read_pickle('analytics_supercurrent_distribution_map.pkl')

fig, ax = plt.subplots(1, figsize=(16,8))
im = ax.imshow( out , cmap='seismic', aspect='auto',vmax=1,vmin=-1, \
          extent=[-501,501,-4.9,4.9])

ax.set_xlabel("$x$  [nm]")
ax.set_ylabel("$\Phi$  [$\Phi_0$]")
cbar = plt.colorbar(im)
cbar.set_label('$J $ $[J_{0}]$ ')
#plt.savefig('analytics_supercurrent_distribution_map.pdf')
plt.show()

# Fig 4(b) red

In [ ]:
%%px --local

# tip in x_tip = 0
par.V_value=1
par.xt=0

In [ ]:
# FP
N=300
B_range=np.linspace(-50e-3,50e-3,N)
out1 = lview.map_async(lambda B :B_I_c(par=par,B=B) , B_range)
out1.wait_interactive()
data1 = out1.get()

 # Fig 4(b) black

In [ ]:
%%px --local

# tip in x_tip = 600 (Outside Junction)
par.V_value=1
par.xt=600

In [ ]:
out2 = lview.map_async(lambda B :B_I_c(par=par,B=B) , B_range)
out2.wait_interactive()
data2 = out2.get()

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,10))
ax1.set_xlabel("$ \Phi $ [$\Phi_0$]")
ax1.set_ylabel('$I_{c}$ [$J_0$W]')
# 50 mT = 4.84 \Phi_0 
lns1=ax1.plot(np.linspace(-4.84,4.84,N),np.array(data2),'k',label='$x_{tip}=600$ $nm$')
lns2=ax1.plot(np.linspace(-4.84,4.84,N),np.array(data1),'r',label='$x_{tip}=0$')
ax1.tick_params(axis='y',labelcolor='k')
plt.ylim(0,1000)
# added these two lines
lns = lns1+lns2
labs = [l.get_label() for l in lns]
plt.legend(lns, labs, loc=0)
plt.xticks(np.arange(-5,5,1))
plt.xlim(-4.84,4.84)
plt.grid()
plt.savefig('fig_4b.pdf')
plt.show()

In [ ]:
%%px --local

def Ic_vs_xtB(par,B,xt): # calculates Ic in function of magnetic Field and x_tip position
    par.xt=xt
    par.B=B
    Ic=[]
    for phi in np.linspace(0,2*np.pi,150):
        par.phi=phi
        value=0
        d=10
        for x in np.arange(-par.W/2,par.W/2,d):
            value+= curr_dist(x,par) *d
            
        Ic.append( value )
        
    return max(Ic)

In [ ]:
learner = adaptive.Learner2D(lambda xtB :Ic_vs_xtB(par, B=xtB[1], xt =  xtB[0]), bounds=[(-par.W/2-100,par.W/2+100),(-0.05,0.05)])
runner = adaptive.Runner(learner, executor=cluster, goal=lambda l: l.loss() < 0.001)
runner.live_info()

 # overwrites data
pd.DataFrame(learner.plot().image.I.data ).to_pickle('analytics_Ic_xtB_map.pkl')    #to save the dataframe, df to *.pkl


In [ ]:
out  = pd.read_pickle('analytics_Ic_xtB_map.pkl')

fig, ax = plt.subplots(1, figsize=(16,8))
im = ax.imshow( out , cmap='inferno', aspect='auto', \
          extent=[-600,600,-4.9,4.9])


ax.set_xlabel("$x_{tip}$  [nm]")
ax.set_ylabel("$\Phi$  [$\Phi_0$]")
cbar = plt.colorbar(im)
cbar.set_label('$I_{c}$ [$J_0$W]')
### to plot lines 
for i in range(-4,5):
    x = [-600, 600]
    t = i*B_period/10.3
    y = [t,t]
    plt.plot(x, y, color="white", linewidth=1.5)
    
plt.plot([0,0],[-4.9,4.9], color="white", linewidth=2.5)
plt.plot([598,598],[-4.9,4.9], color="white", linewidth=1.1)
for i in [-500,500]:
    y = [-4.9, 4.9]
    t = i
    x = [t,t]
    plt.plot(x, y, '--w', linewidth=1.1)

###
#plt.savefig("Fig4b.pdf")
plt.show()

# Supercurrent distribution FIG 5(a)

In [ ]:
par.V_value=1
par.phi=1*np.pi/2
par.B=B_period*1e-3*.5

values=[]
x_tab = np.arange(-par.W/2-1,par.W/2+1,1)
for x in x_tab:
    values.append(curr_dist(x,par) )
plt.figure(figsize=(10,6))
plt.plot(x_tab,np.array(values)/J_max,'k',linewidth=4)

plt.xlabel("$ x $ [nm]")
plt.ylabel(" J $[J_0]$")
plt.grid()
plt.ylim(-1,1)
plt.xlim(-501,501)
#plt.savefig('fig3a.pdf')
plt.show()

# FIG 5(b)

In [ ]:
par.V_value=1
par.phi=-1*np.pi/2
par.B=B_period*1e-3

values=[]
x_tab = np.arange(-par.W/2-1,par.W/2+1,1)
for x in x_tab:
    values.append(curr_dist(x,par) )
plt.figure(figsize=(10,6))
plt.plot(x_tab,np.array(values)/J_max,'k',linewidth=4)

plt.xlabel("$ x $ [nm]")
plt.ylabel(" J $[J_0]$")
plt.grid()
plt.ylim(-1,1)
plt.xlim(-501,501)
#plt.savefig('fig3a.pdf')
plt.show()

 # FIG 5(c)

In [ ]:
par.xt=200
par.V_value=1
par.phi=-0.9*np.pi
par.B=B_period*1e-3

values=[]
x_tab = np.arange(-par.W/2-1,par.W/2+1,1)
for x in x_tab:
    values.append(curr_dist(x,par) )
plt.figure(figsize=(10,6))
plt.plot(x_tab,np.array(values)/J_max,'k',linewidth=4)

plt.xlabel("$ x $ [nm]")
plt.ylabel(" J $[J_0]$")
plt.grid()
plt.ylim(-1,1)
plt.xlim(-501,501)
#plt.savefig('fig_e.pdf')

 # FIG 5(d)

In [ ]:
par.xt=0
par.V_value=1
par.phi=-np.pi/2
par.B=B_period*1e-3*1.5

values=[]
x_tab = np.arange(-par.W/2-1,par.W/2+1,1)
for x in x_tab:
    values.append(curr_dist(x,par) )
plt.figure(figsize=(10,6))
plt.plot(x_tab,np.array(values)/J_max,'k',linewidth=4)

plt.xlabel("$ x $ [nm]")
plt.ylabel(" J $[J_0]$")
plt.grid()
plt.ylim(-1,1)
plt.xlim(-501,501)

#plt.savefig('fig3c.pdf')
plt.show()

 # FIG 5(e)

In [ ]:
par.xt=0
par.V_value=1
par.phi=np.pi/2
par.B=B_period*1e-3 *2.5

values=[]
x_tab = np.arange(-par.W/2-1,par.W/2+1,1)
for x in x_tab:
    values.append(curr_dist(x,par) )
plt.figure(figsize=(10,6))
plt.plot(x_tab,np.array(values)/J_max,'k',linewidth=4)

plt.xlabel("$ x $ [nm]")
plt.ylabel(" J $[J_0]$")
plt.grid()
plt.ylim(-1,1)
plt.xlim(-501,501)
#plt.savefig('fig3f.pdf')
plt.show()

# FIG 6(a)

In [ ]:
%%px --local

par.V_value=1
par.B=B_period*1e-3 *1.5
xt_range=np.linspace(-600,600,200)

# $\Phi = 1.5 \Phi_0$

In [ ]:
out2 = lview.map_async(lambda xt :Ic_vs_xtB(par=par,B=B_period*1.5e-3,xt=xt) , xt_range)
out2.wait_interactive()
data = out2.get()
temp = Ic_vs_xtB(par=par,B=B_period*1.5e-3,xt=1e6) # Criticalcurrent when there is no tip
data = np.array(data)- temp

# $\Phi =  \Phi_0$

In [ ]:
out2 = lview.map_async(lambda xt :Ic_vs_xtB(par=par,B=B_period*1e-3,xt=xt) , xt_range)
out2.wait_interactive()
data2 = out2.get()
temp2 = Ic_vs_xtB(par=par,B=B_period*1e-3,xt=1e6) # Criticalcurrent when there is no tip
data2 = np.array(data2)- temp2

In [ ]:
fig, ax = plt.subplots(figsize=[16, 8])
plt.plot(xt_range,np.array(data2),'r',label = '$\Phi =  \Phi_0$')
plt.plot(xt_range,np.array(data),'k',label = '$\Phi = 1.5\Phi_0$')

plt.xlabel("$ x_{tip} $ [nm]")
plt.ylabel("$I_c - I_{c0}$ [$J_0 $]")
plt.grid()
plt.xlim(-600,600)
plt.legend()
#plt.savefig("Ic_vs_xtip_a.pdf")
plt.show()

In [ ]:
%%px --local

def phi_vs_xtB(par,B,xt): # returns phi, that gives critical current in function of B,x_tip
    par.xt=xt
    par.B=B
    Ic=[]
    phitab= np.linspace(-np.pi,np.pi,100)
    for phi in phitab:
        par.phi=phi
        value=0
        d=10
        for x in np.arange(-par.W/2,par.W/2,d):
            value+= curr_dist(x,par) *d
        Ic.append( value )
    return phitab[np.argmax(Ic)]

 # FIG 6(b)

In [ ]:
learner = adaptive.Learner2D(lambda xtB :phi_vs_xtB(par, B=xtB[1], xt =  xtB[0]), bounds=[(-par.W/2-100,par.W/2+100),(-0.05,0.05)])
runner = adaptive.Runner(learner, executor=cluster, goal=lambda l: l.loss() < 0.02)
runner.live_info()

 # overwrites data
pd.DataFrame(learner.plot().image.I.data ).to_pickle('Phase_difference_versus_flux_and_the_tip_position.pkl')    #to save the dataframe, df to *.pkl


In [ ]:
out  = pd.read_pickle('Phase_difference_versus_flux_and_the_tip_position.pkl')

fig, ax = plt.subplots(1, figsize=(16,8))
im = ax.imshow( out/np.pi , cmap='seismic', aspect='auto',vmax=1,vmin=-1, \
          extent=[-600,600,-4.9,4.9])

ax.set_xlabel("$x_{tip}$  [nm]")
ax.set_ylabel("$\Phi$  [$\Phi_0$]")
cbar = plt.colorbar(im)
cbar.set_label('$\phi$  $[\pi]$ ')
### to plot lines
for i in range(-4,5):
    x = [-600, 600]
    t = i*B_period/10.3
    y = [t,t]
    plt.plot(x, y, color="white", linewidth=1.1)
###
#plt.savefig('Phase_difference_versus_flux_and_the_tip_position.pdf')
plt.show()

 # New analytics with asymmetry

In [ ]:
%%px --local

def curr_dist(x,par):
    return J_S0(x,par) * np.sin(gamma(x,par))/np.abs( np.cos(gamma(x,par)/2 ))

 # FIG 8(b)

In [ ]:
learner = adaptive.Learner1D(lambda xt :Ic_vs_xtB(par, B=1.5e-3 * B_period , xt =  xt), bounds=(-par.W/2-100,par.W/2+100))
runner = adaptive.Runner(learner, executor=cluster, goal=lambda l: l.loss() < 0.001)
runner.live_info()

 
 # overwrites data
pd.DataFrame(learner.plot().scatter.I.data ).to_pickle('analytics_shift_new.pkl')    #to save the dataframe, df to *.pkl



In [ ]:
%%px --local
par.xt = 1e6

In [ ]:
a = 436.261260088964 #  B_I_c(par, B=1.5e-3 * B_period )
data = pd.read_pickle('analytics_shift_new.pkl')

plt.subplots(1, figsize=(16,8))
plt.plot(data['x'],data['y']-a, "k",label ="semiclassical")
plt.legend()
plt.xlim(-600,600)
plt.xlabel('$x_{tip}$ $[nm]$')
plt.ylabel('$I_{c}$-$I_{c0}$ [au]')
plt.grid()
plt.show()

In [ ]:
learner = adaptive.Learner2D(lambda xtB :Ic_vs_xtB(par, B=xtB[1], xt =  xtB[0]), bounds=[(-par.W/2-100,par.W/2+100),(-0.05,0.05)])
runner = adaptive.Runner(learner, executor=cluster, goal=lambda l: l.loss() < 0.001)
runner.live_info()

 # overwrites data
pd.DataFrame(learner.plot().image.I.data ).to_pickle('analytics_Ic_xtB_shift_map.pkl')    #to save the dataframe, df to *.pkl





In [ ]:
out  = pd.read_pickle('analytics_Ic_xtB_shift_map.pkl')

fig, ax = plt.subplots(1, figsize=(16,8))
im = ax.imshow( out , cmap='inferno', aspect='auto', \
          extent=[-600,600,-4.9,4.9])


ax.set_xlabel("$x_{tip}$  [nm]")
ax.set_ylabel("$\Phi$  [$\Phi_0$]")
cbar = plt.colorbar(im)
cbar.set_label('$I_c$ [$au$]')
### to plot lines
for i in range(-4,5):
    x = [-600, 600]
    t = i*B_period/10.3
    y = [t,t]
    plt.plot(x, y, color="white", linewidth=1.5)
    for i in [-500,500]:
    y = [-4.9, 4.9]
    t = i
    x = [t,t]
    plt.plot(x, y, '--w', linewidth=1.1)
    ###
#plt.savefig("fig.pdf")
plt.show()